In [1]:
import statsmodels.api as sm
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib widget

In [11]:
# Load the Excel file into a DataFrame
NrSubtype='L1' #'L1' or 'All'
analysisfile='Spindles_ABdetection_GrandGlobalAB'
directory="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording/Analysis_Spdl_S1/"
combined_df = pd.read_excel(f'{directory}{NrSubtype}_{analysisfile}.xlsx', index_col=0)            


analysisfile2='Spindles_ABdetection_GrandAverageAB'
directory="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording/Analysis_Spdl_S1/"
excel_file = f'{directory}{NrSubtype}_{analysisfile2}.xlsx'
excel_data = pd.read_excel(excel_file, sheet_name=None, index_col=0, header=None) 
dict_Spindles_average = {}
for sheet_name, sheet_data in excel_data.items():
    dict_Spindles_average[sheet_name] = sheet_data


analysisfile3='Spindles_ABdetection_SpikeGrandAverageAB'
directory="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording/Analysis_Spdl_S1/"
excel_file = f'{directory}{NrSubtype}_{analysisfile3}.xlsx'
excel_data = pd.read_excel(excel_file, sheet_name=None, index_col=0, header=None) 
dict_Spindles_Spikeaverage = {}
for sheet_name, sheet_data in excel_data.items():
    dict_Spindles_Spikeaverage[sheet_name] = sheet_data

In [12]:
def clean_string(s):
    return s.replace("[", "").replace("]", "").replace("'", "")

# Apply the function to the specific column
combined_df['SpdlStatut'] = combined_df['SpdlStatut'].apply(clean_string)

In [13]:
combined_df

,Mice,Session,Session_Time,Unique_Unit,UnitNumber,UnitValue,SpdlStatut,SpdlNumber,SpdlDuration (ms),SWR inside Spdl,...,CalciumActivityAfter,AUC_calciumBefore,AUC_calciumAfter,SpikeActivityPreference,SpikeActivityBefore,SpikeActivityAfter,AUC_spikeBefore,AUC_spikeAfter,Unit_ID,Spdl_ID
0,BlackLinesOK,session1,NaN,3,0,3,UnCoupled,0,629.0,False,...,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,BlackLinesOK3,BlackLinesOKsession10
1,BlackLinesOK,session1,NaN,3,0,3,UnCoupled,1,1030.0,False,...,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,BlackLinesOK3,BlackLinesOKsession11
2,BlackLinesOK,session1,NaN,3,0,3,UnCoupled,2,1049.0,False,...,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,BlackLinesOK3,BlackLinesOKsession12
3,BlackLinesOK,session1,NaN,3,0,3,UnCoupled,3,524.0,False,...,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,BlackLinesOK3,BlackLinesOKsession13
4,BlackLinesOK,session1,NaN,3,0,3,UnCoupled,4,1340.0,False,...,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,BlackLinesOK3,BlackLinesOKsession14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526,RedLinesOK,session5,NaN,75,8,108,UnCoupled,9,773.0,False,...,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,RedLinesOK75,RedLinesOKsession59
2527,RedLinesOK,session5,NaN,75,8,108,UnCoupled,10,872.0,False,...,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,RedLinesOK75,RedLinesOKsession510
2528,RedLinesOK,session5,NaN,75,8,108,UnCoupled,11,499.0,False,...,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,RedLinesOK75,RedLinesOKsession511
2529,RedLinesOK,session5,NaN,75,8,108,UnCoupled,12,674.0,False,...,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.0,0.0,RedLinesOK75,RedLinesOKsession512


In [17]:
Spdldurmean = combined_df.groupby('Spdl_ID')['SpdlDuration (ms)'].mean()
filenameOut = f'{directory}{NrSubtype}_SpdlPFCdurations_AB.xlsx'
writer = pd.ExcelWriter(filenameOut)
Spdldurmean.to_excel(writer)
writer.close()
np.mean(Spdldurmean)


1037.3505535055351

AUC calcium

In [106]:
# Create df for All Spdl
AUC_calciumBefore_perUnit = combined_df.groupby('Unit_ID')['AUC_calciumBefore'].mean()
AUC_calciumAfter_perUnit = combined_df.groupby('Unit_ID')['AUC_calciumAfter'].mean()
allAUCresult = pd.concat([AUC_calciumBefore_perUnit,AUC_calciumAfter_perUnit], axis=1)

# Create dfs for each Spdl Statut
SpdlStatut_AUC_calciumBefore_perUnit = combined_df.pivot_table(index='Unit_ID', columns='SpdlStatut', values='AUC_calciumBefore', aggfunc='mean')
SpdlStatutNames=SpdlStatut_AUC_calciumBefore_perUnit.columns.tolist()
new_columns = [f'Before_{col}' for col in SpdlStatut_AUC_calciumBefore_perUnit.columns]
SpdlStatut_AUC_calciumBefore_perUnit.columns = new_columns
SpdlStatut_AUC_calciumAfter_perUnit = combined_df.pivot_table(index='Unit_ID', columns='SpdlStatut', values='AUC_calciumAfter', aggfunc='mean')
new_columns = [f'After_{col}' for col in SpdlStatut_AUC_calciumAfter_perUnit.columns]
SpdlStatut_AUC_calciumAfter_perUnit.columns = new_columns
# Combine each Spdl Statut df into one
AUC_interleaved_cols = [col for pair in zip(SpdlStatut_AUC_calciumBefore_perUnit.columns, SpdlStatut_AUC_calciumAfter_perUnit.columns) for col in pair]
AUCresult = pd.concat([SpdlStatut_AUC_calciumBefore_perUnit[col] if col in SpdlStatut_AUC_calciumBefore_perUnit.columns else SpdlStatut_AUC_calciumAfter_perUnit[col] for col in AUC_interleaved_cols], axis=1)

# Create a dict containing each Spdl Statut and All Spdl
AUCdata_dict = {}
c=0
for i in range(len(SpdlStatutNames)):
    columns = AUCresult.columns[c:c+2]
    c=c+2
    AUCdata_dict[SpdlStatutNames[i]] = AUCresult[columns].values.tolist()
AUCdata_dict['AllSpdl']=allAUCresult.values.tolist()


AUCdata_dict_ActiveOnly={}
for key in AUCdata_dict:
    # Filter out rows with non-zero values
    filtered_rows = [row for row in AUCdata_dict[key] if any(val != 0 for val in row) and np.any(~np.isnan(row))]
    # Update the key with filtered rows
    AUCdata_dict_ActiveOnly[key] = filtered_rows


filenameOut = f'{directory}{NrSubtype}_SpdlPFC_AUC_perUnitAB.xlsx'
with pd.ExcelWriter(filenameOut) as writer:
    # Iterate over each key-value pair in the dictionary
    for sheet_name, data_list in AUCdata_dict.items():
        # Convert the list to a DataFrame
        df = pd.DataFrame(data_list, columns=['Before', 'After'])
        # Write the DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

filenameOut = f'{directory}{NrSubtype}_SpdlPFC_AUC_perUnit_Active OnlyAB.xlsx'
with pd.ExcelWriter(filenameOut) as writer:
    # Iterate over each key-value pair in the dictionary
    for sheet_name, data_list in AUCdata_dict_ActiveOnly.items():
        # Convert the list to a DataFrame
        df = pd.DataFrame(data_list, columns=['Before', 'After'])
        # Write the DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

Spike Activity 

In [107]:
# Create df for All Spdl
SpikeActivityBefore_perUnit = combined_df.groupby('Unit_ID')['SpikeActivityBefore'].mean()
SpikeActivityAfter_perUnit = combined_df.groupby('Unit_ID')['SpikeActivityAfter'].mean()
allSPresult = pd.concat([SpikeActivityBefore_perUnit,SpikeActivityAfter_perUnit], axis=1)

# Create dfs for each Spdl Statut
SpdlStatut_SpikeActivityBefore_perUnit = combined_df.pivot_table(index='Unit_ID', columns='SpdlStatut', values='SpikeActivityBefore', aggfunc='mean')
SpdlStatutNames=SpdlStatut_SpikeActivityBefore_perUnit.columns.tolist()
new_columns = [f'Before_{col}' for col in SpdlStatut_SpikeActivityBefore_perUnit.columns]
SpdlStatut_SpikeActivityBefore_perUnit.columns = new_columns
SpdlStatut_SpikeActivityAfter_perUnit = combined_df.pivot_table(index='Unit_ID', columns='SpdlStatut', values='SpikeActivityAfter', aggfunc='mean')
new_columns = [f'After_{col}' for col in SpdlStatut_SpikeActivityAfter_perUnit.columns]
SpdlStatut_SpikeActivityAfter_perUnit.columns = new_columns
# Combine each Spdl Statut df into one
SP_interleaved_cols = [col for pair in zip(SpdlStatut_SpikeActivityBefore_perUnit.columns, SpdlStatut_SpikeActivityAfter_perUnit.columns) for col in pair]
SPresult = pd.concat([SpdlStatut_SpikeActivityBefore_perUnit[col] if col in SpdlStatut_SpikeActivityBefore_perUnit.columns else SpdlStatut_SpikeActivityAfter_perUnit[col] for col in SP_interleaved_cols], axis=1)


# Create a dict containing each Spdl Statut and All Spdl
SPdata_dict = {}
c=0
for i in range(len(SpdlStatutNames)):
    columns = SPresult.columns[c:c+2]
    c=c+2
    SPdata_dict[SpdlStatutNames[i]] = SPresult[columns].values.tolist()
SPdata_dict['AllSpdl']=allSPresult.values.tolist()

SPdata_dict_ActiveOnly={}
for key in SPdata_dict:
    # Filter out rows with non-zero values
    filtered_rows = [row for row in SPdata_dict[key] if any(val != 0 for val in row) and np.any(~np.isnan(row))]
    # Update the key with filtered rows
    SPdata_dict_ActiveOnly[key] = filtered_rows


filenameOut = f'{directory}{NrSubtype}_SpdlPFC_SpikeAct_perUnitAB.xlsx'
with pd.ExcelWriter(filenameOut) as writer:
    # Iterate over each key-value pair in the dictionary
    for sheet_name, data_list in SPdata_dict.items():
        # Convert the list to a DataFrame
        df = pd.DataFrame(data_list, columns=['Before', 'After'])
        # Write the DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

filenameOut = f'{directory}{NrSubtype}_SpdlPFC_SpikeAct_perUnit_Active OnlyAB.xlsx'
with pd.ExcelWriter(filenameOut) as writer:
    # Iterate over each key-value pair in the dictionary
    for sheet_name, data_list in SPdata_dict_ActiveOnly.items():
        # Convert the list to a DataFrame
        df = pd.DataFrame(data_list, columns=['Before', 'After'])
        # Write the DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)
